In [18]:
!pip install pdfplumber

  Using cached pillow-10.4.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------  5.5/5.6 MB 27.9 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 24.4 MB/s eta 0:00:00
Using cached pillow-10.4.0-cp311-cp311-win_amd64.whl (2.6 MB)
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --------------------- ------------------ 1.6/2.9 MB 8.3 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 6.0 MB/s eta 0:00:00
Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl (99 kB)
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.1 M

In [3]:
!pip install pypdf2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [26]:
!pip install langchain

  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp311-none-win_amd64.whl.metadata (6.7 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.met

In [58]:
import PyPDF2

def extract_text_with_structure(pdf_path, start_page=8, max_pages=100):
    docs = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        total_pages = len(pdf_reader.pages)
        end_page = min(start_page + max_pages, total_pages)
        
        for page_num in range(start_page, end_page):  # Adjusting start_page to zero-indexed
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text() or ""  # Extract text and handle None if no text is extracted
            
            if page_text:
                # Split text by lines to preserve structure and avoid adding extra spaces
                page_lines = page_text.splitlines()
                page_text_preserved = "\n".join(line.strip() for line in page_lines if line.strip())

                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=800,
                    chunk_overlap=50,
                    separators=["\n\n", "\n"]  # Keep line breaks as separators
                )
                # Split the text into chunks while preserving line breaks
                texts = text_splitter.create_documents([page_text_preserved])
                for text in texts:
                    text.metadata = {"Page number": page_num - 7}
                    docs.append(text)
                
                # Print or use the text chunks as needed
                    
    
    return docs

docs = extract_text_with_structure('Introduction-to-Robotics-3rd-edition.pdf', start_page=8, max_pages=100)
# print(docs)
# print(len(docs))
# print(pdf_text_with_structure)
docs

[Document(metadata={'Page number': 1}, page_content='CHAPTER 1Introduction\n1.1BACKGROUND1.2THE MECHANICS AND CONTROL OF MECHANICAL MANIPULATORS1.3NOTATION'),
 Document(metadata={'Page number': 1}, page_content='\n1.1BACKGROUNDThehistory of industrial automation is characterized by periods of rapid change inpopular methods. Either as a cause or, perhaps, an effect, such periods of change inautomation techniques seem closely tied to world economics. Use of the industrialrobot, which became identifiable as a unique device in the 1960s [1], along withcomputer-aided design (CAD) systems and computer-aided manufacturing (CAM)systems, characterizes the latest trends in the automation of the manufacturingprocess. These technologies are leading industrial automation through anothertransition, the scope of which is stifi unknown [2].In North America, there was much adoption of robotic equipment in the early1980s, followed by a brief pull-back in the late 1980s. Since that time, the market hasbe

In [ ]:
!pip install langchain_pinecone

In [ ]:
!pip install langchain_openai

In [59]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

os.environ['OPENAI_API_KEY'] = ''
os.environ['PINECONE_API_KEY'] = '34e66b6f-03f5-420d-95e5-a4fa9560977b'

index_name = "e-book"
embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")

vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    index_name=index_name,
    embedding=embeddings
)


In [61]:
pinecone_api_key = '34e66b6f-03f5-420d-95e5-a4fa9560977b'
index_name = "e-book"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
pinecone = PineconeVectorStore(index_name=index_name, embedding=embeddings, pinecone_api_key=pinecone_api_key)  

In [64]:
retriever = pinecone.as_retriever(
                # search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
            )
retriever.invoke("what is tranformation matrix?")

[Document(id='4a6286ad-3c5a-4afe-a483-acd39eb4a2c9', metadata={'Page number': 33.0}, page_content='Section 2.4Operators: translations, rotations, and transformations 33Transformation operatorsAswith vectors and rotation matrices, a frame has another interpretation asa transformation operator. In this interpretation, only one coordinate system isinvolved, and so the symbol T is used without sub- or superscripts. The operator Trotates and translates a vector A P1 to computea new vector,AP_TAP (2.33)Again, as in the case of rotations, the mathematics described in (2.18) and in (2.33)is the same, only our interpretation is different. This fact also allows us to see howto obtain homogeneous transforms that are to be used as operators:The transform that rotates by R and translates by Q isthe same as the transformthat describes afraine rotated by Rand translated by Q relativeto the reference frame.A transform is usually thought of as being in the form of a homogeneoustransform with general ro

In [21]:
type(pdf_text_with_structure)

str